# Motion Planning

In [29]:
# Grid format:
#   0 = Navigable space
#   1 = Occupied space


grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 0]]
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

## Search algorithm iteration

In [114]:
# get expansion grid for search
def search(grid,init,goal,cost):
    rows = len(grid)
    cols = len(grid[0])    
    checked = [[0]*cols for _ in range(rows)]
    expand = [[-1]*cols for _ in range(rows)]

    open = [[0, init[0], init[1]]]
    count = 0
    
    while len(open) > 0:
        # take an item from open list
        open.sort()
        open.reverse()
        take = open.pop()
        expand[take[1]][take[2]] = count
        count += cost
        checked[take[1]][take[2]] = 1
        if take[1] == goal[0] and take[2] == goal[1]:
            return expand

        # expand
        for move in delta:
            n = [take[0]+cost, take[1]+move[0], take[2]+move[1]]
            if 0 <= n[1] < rows and 0 <= n[2] < cols: # within the grid
                if grid[n[1]][n[2]] == 0: # unobstructed space
                    if checked[n[1]][n[2]] == 0: # not checked already
                        open.append(n)
                        checked[n[1]][n[2]] = 1
    return expand
expansion = search(grid, init, goal, cost)
expansion

[[0, 1, -1, 14, 17, 19],
 [2, 3, -1, 12, 15, 18],
 [4, 5, 7, 10, -1, 20],
 [6, 8, -1, -1, -1, 21],
 [9, 11, 13, 16, -1, 22]]

In [115]:
# get a list of costs that lead to path
rows = len(grid)
cols = len(grid[0])

smallest_l = []
if expansion[goal[0]][goal[1]] != -1:
    current = goal
    smallest_l = [expansion[goal[0]][goal[1]]]
    
    while current != init:
        for move in delta:
            n = [current[0]+move[0], current[1]+move[1]]
            if 0 <= n[0] < rows and 0 <= n[1] < cols: # within the grid
                if expansion[n[0]][n[1]] != -1: # unobstructed space
                    if expansion[n[0]][n[1]] < expansion[current[0]][current[1]]:
                        latest = [n[0], n[1]]
        
        smallest_l.append(expansion[latest[0]][latest[1]])
        current = latest

smallest_l.reverse()
smallest_l

[0, 2, 4, 5, 7, 10, 12, 15, 18, 20, 21, 22]

In [116]:
# generate path from costs
path = [[' ']*cols for _ in range(rows)]
path[goal[0]][goal[1]] = '*'

current = init
for i in range(len(smallest_l)-1):
    for j, move in enumerate(delta):
        n = [current[0]+move[0], current[1]+move[1]]
        if 0 <= n[0] < rows and 0 <= n[1] < cols: # within the grid
            if expansion[n[0]][n[1]] == smallest_l[i+1]:
                path[current[0]][current[1]] = delta_name[j]
                current = n
path

[['v', ' ', ' ', ' ', ' ', ' '],
 ['v', ' ', ' ', '>', '>', 'v'],
 ['>', '>', '>', '^', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', '*']]

In [146]:
# better algorithm 
# get expansion grid for search
def search(grid,init,goal,cost):
    rows = len(grid)
    cols = len(grid[0])    
    checked = [[0]*cols for _ in range(rows)]
    actions = [[-1]*cols for _ in range(rows)]
    
    x = init[0]
    y = init[1]
    g = 0
    
    open = [[g,x,y]]
    checked[x][y] = 1
    
    while len(open) > 0:
        # take an item from open list
        open.sort()
        open.reverse()
        take = open.pop()
        x = take[1]
        y = take[2]
        g = take[0]
        if x == goal[0] and y == goal[1]:
            break

        # expand
        for i, move in enumerate(delta):
            x2 = x + move[0]
            y2 = y + move[1]
            if 0 <= x2 < rows and 0 <= y2 < cols: # within the grid
                if grid[x2][y2] == 0 and checked[x2][y2] == 0: # unobstructed space and not checked already
                    g2 = g + cost
                    open.append([g2,x2,y2])
                    checked[x2][y2] = 1
                    actions[x2][y2] = i
                  
    # generate policy
    policy = [[' ']*cols for _ in range(rows)]
    x = goal[0]
    y = goal[1]
    policy[x][y] = '*'
    while x != init[0] and y != init[1]:
        x2 = x - delta[actions[x][y]][0]
        y2 = y - delta[actions[x][y]][1]
        policy[x2][y2] = delta_name[actions[x][y]]
        x = x2
        y = y2
    policy[init[0]][init[1]] = delta_name[actions[x][y]]
    return policy
    
policy = search(grid, init, goal, cost)
policy

[['>', 'v', ' ', ' ', ' ', ' '],
 [' ', 'v', ' ', '>', '>', 'v'],
 [' ', '>', '>', '^', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', '*']]

## A* Search

In [208]:
grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]]
heuristic = [[9, 8, 7, 6, 5, 4],
             [8, 7, 6, 5, 4, 3],
             [7, 6, 5, 4, 3, 2],
             [6, 5, 4, 3, 2, 1],
             [5, 4, 3, 2, 1, 0]]

init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

In [209]:
# expansion
def search(grid,init,goal,cost,heuristic):
    closed = [[0 for col in range(len(grid[0]))] for row in range(len(grid))]
    closed[init[0]][init[1]] = 1

    expand = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    action = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]

    x = init[0]
    y = init[1]
    g = 0
    f = g + heuristic[x][y]

    open = [[f, g, x, y]]

    found = False  # flag that is set when search is complete
    resign = False # flag set if we can't find expand
    count = 0
    
    while not found and not resign:
        if len(open) == 0:
            resign = True
            return "Fail"
        else:
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[2]
            y = next[3]
            f = next[0]
            g = next[1]
            expand[x][y] = count
            count += 1
            
            if x == goal[0] and y == goal[1]:
                found = True
            else:
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >=0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            g2 = g + cost
                            f2 = g2 + heuristic[x2][y2]
                            open.append([f2, g2, x2, y2])
                            closed[x2][y2] = 1

    return expand
search(grid, init, goal, cost, heuristic)

[[0, -1, -1, -1, -1, -1],
 [1, -1, -1, -1, -1, -1],
 [2, -1, -1, -1, -1, -1],
 [3, -1, 8, 9, 10, 11],
 [4, 5, 6, 7, -1, 12]]

## Dynamic Programming for path planning

In [210]:
grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1 # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

In [211]:
def compute_value(grid,goal,cost):
    # initialize value function
    value = [[99 for col in range(len(grid[0]))] for row in range(len(grid))]
    closed = [[0 for col in range(len(grid[0]))] for row in range(len(grid))]
    closed[goal[0]][goal[1]] = 1
    
    open = [[0, goal[0], goal[1]]]
    while len(open) > 0:
        open.sort()
        open.reverse()
        next = open.pop()
        x = next[1]
        y = next[2]
        v = next[0]
        value[x][y] = v
        for move in delta:
            x2 = x + move[0]
            y2 = y + move[1]
            v2 = v + cost
            if x2 >= 0 and x2 < len(grid) and y2 >=0 and y2 < len(grid[0]):
                if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                    open.append([v2, x2, y2])
                    closed[x2][y2] = 1
    return value 
compute_value(grid, goal, cost)

[[11, 99, 7, 6, 5, 4],
 [10, 99, 6, 5, 4, 3],
 [9, 99, 5, 4, 3, 2],
 [8, 99, 4, 3, 2, 1],
 [7, 6, 5, 4, 99, 0]]

In [212]:
grid = [[0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0]]
goal = [len(grid)-1, len(grid[0])-1]
compute_value(grid, goal, cost)

[[99, 99, 99, 99, 5, 4],
 [99, 99, 99, 99, 4, 3],
 [99, 99, 99, 99, 3, 2],
 [99, 99, 99, 99, 2, 1],
 [99, 99, 99, 99, 1, 0]]

In [213]:
grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]]
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1 # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

In [214]:
def optimum_policy(grid,goal,cost):
    value = [[99 for row in range(len(grid[0]))] for col in range(len(grid))]
    policy = [[' ' for col in range(len(grid[0]))] for row in range(len(grid))]
    change = True

    while change:
        change = False

        for x in range(len(grid)):
            for y in range(len(grid[0])):
                if goal[0] == x and goal[1] == y:
                    if value[x][y] > 0:
                        value[x][y] = 0
                        policy[x][y] = '*'
                        change = True

                elif grid[x][y] == 0:
                    for a in range(len(delta)):
                        x2 = x + delta[a][0]
                        y2 = y + delta[a][1]

                        if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]) and grid[x2][y2] == 0:
                            v2 = value[x2][y2] + cost

                            if v2 < value[x][y]:
                                policy[x][y] = delta_name[a]
                                change = True
                                value[x][y] = v2

    return policy
optimum_policy(grid, goal, cost)

[['v', ' ', 'v', 'v', 'v', 'v'],
 ['v', ' ', 'v', 'v', 'v', 'v'],
 ['v', ' ', 'v', 'v', 'v', 'v'],
 ['v', ' ', '>', '>', '>', 'v'],
 ['>', '>', '^', '^', ' ', '*']]

## 2D grid policy

You are given a car in grid with initial state init. Your task is to compute and return the car's optimal path to the position specified in goal. 

The costs for each motion are as defined in cost.

There are four motion directions: up, left, down, and right. Increasing the index in this array corresponds to making a left turn, and decreasing the index corresponds to making a right turn.

In [108]:
forward = [[-1,  0], # go up
           [ 0, -1], # go left
           [ 1,  0], # go down
           [ 0,  1]] # go right
forward_name = ['up', 'left', 'down', 'right']

# action has 3 values: right turn, no turn, left turn
action = [-1, 0, 1]
action_name = ['R', '#', 'L']

# EXAMPLE INPUTS:
# grid format:
#     0 = navigable space
#     1 = unnavigable space 
grid = [[1, 1, 1, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1, 1]]

init = [4, 3, 0] # given in the form [row,col,direction]
                 # direction = 0: up
                 #             1: left
                 #             2: down
                 #             3: right
                
goal = [2, 0] # given in the form [row,col]

cost = [2, 1, 20] # cost has 3 values, corresponding to making 
                  # a right turn, no turn, and a left turn

# EXAMPLE OUTPUT:
# calling optimum_policy2D with the given parameters should return 
# [[' ', ' ', ' ', 'R', '#', 'R'],
#  [' ', ' ', ' ', '#', ' ', '#'],
#  ['*', '#', '#', '#', '#', 'R'],
#  [' ', ' ', ' ', '#', ' ', ' '],
#  [' ', ' ', ' ', '#', ' ', ' ']]
# ----------

In [110]:
def optimum_policy2D(grid,init,goal,cost):
    value = [[[999 for k in range(len(grid[0]))] for j in range(len(grid))] for i in range(4)]
    policy = [[[' ' for k in range(len(grid[0]))] for j in range(len(grid))] for i in range(4)]
    changed = True
    
    while changed:
        changed = False
        for x in range(len(grid)):
            for y in range(len(grid[0])):
                for orientation in range(len(forward)):
                    if goal[0] == x and goal[1] == y:
                        if value[orientation][x][y] > 0:
                            value[orientation][x][y] = 0
                            policy[orientation][x][y] = '*'
                            changed = True
                    elif grid[x][y] == 0:
                        # for each different action
                        for i in range(len(action)):
                            o2 = (orientation + action[i]) % 4
                            x2 = x + forward[o2][0]
                            y2 = y + forward[o2][1]
                            
                            if 0 <= x2 < len(grid) and 0 <= y2 < len(grid[0]) and grid[x2][y2] == 0:
                                v2 = value[o2][x2][y2] + cost[i]
                                if v2 < value[orientation][x][y]:
                                    value[orientation][x][y] = v2
                                    policy[orientation][x][y] = action_name[i]
                                    changed = True

    # make 2d policy (depends on initial condition)
    # go though the policy from goal and set it up based on 3d policy
    policy2d = [[' ' for k in range(len(grid[0]))] for j in range(len(grid))]
    x = init[0]
    y = init[1]
    orientation = init[2]
    policy2d[x][y] = policy[orientation][x][y]
    while policy[orientation][x][y] != '*':
        if policy[orientation][x][y] == '#':
            o2 = orientation
        elif policy[orientation][x][y] == 'R':
            o2 = (orientation - 1) % 4
        elif policy[orientation][x][y] == 'L':
            o2 = (orientation + 1) % 4
        x = x + forward[o2][0]
        y = y + forward[o2][1]
        orientation = o2
        policy2d[x][y] = policy[orientation][x][y]
    return policy2d
optimum_policy2D(grid, init, goal, cost)

[[' ', ' ', ' ', 'R', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', '#'],
 ['*', '#', '#', '#', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', ' '],
 [' ', ' ', ' ', '#', ' ', ' ']]